# Preamble

This solution is not obvious or intuitive, and this idea (or another solution)
You would build an algorithm by understanding the problem and working through the route-finding process on paper. As you work through a solution, you understand what information you need to track. You then think about what data structure you need to store the information. 

As an example of an alternative solution, you may recognise that the
station and lines could be stored as a graph. Then through some 
research (google searching), you discover Dirjkisk Algorithm. This algorithm is a possible solution and would require you to use a 'graph' data structure and change how we store stations and lines. 

In our case we have gone with the work it out by hand with some
hints from the lecturer.

> The following class and method are not complete. It provides additional code and methods to add to your existing solution. Read over the code and determine if and where you need to include the code.

In [ ]:
class Subway():
    """The Subway class represents a subway line with stations 
    and connections bewteen those stations"""

    def __init__(self):
        """Every Subway object has a colleciton of stations and a 
        list of connecitons"""
        self.stations = []  # Use a list as a basic database of stations
        self.connections = []  # List of the direction staitons are connected
        # Added to store a list of neighbourhood network of station
        self.network = {}

    # Discovered/Added when creating SubwayLoader class.
    def add_connection(self, name1, name2, line):
        """Add a connection going in both directions to the subway as 
        long as specified names reference an existing station"""
        if self.has_station(name1) and self.has_station(name2):
            station1 = Station(name1)
            station2 = Station(name2)
            connection1 = Connection(station1, station2, line)
            connection2 = Connection(station2, station1, line)
            if connection1 not in self.connections:
                self.connections.append(connection1)
                # Added next line. Update the neighbourhood network
                self.add_to_network(station1, station2)
            if connection2 not in self.connections:
                self.connections.append(connection2)
                # Added next line. Update the neighbourhood network
                self.add_to_network(station2, station1)

    # To print the directions, we need a Connection object. A connection
    # object provides the direction of travel and the current line.
    # With this infromation, we can print which station to travel to
    # and if we are at a intersection, which line to change to. The way
    # we do this is in the method get_direciton()
    def get_connection(self, station1, station2):
        """Returns a connection if exists, else return None"""
        for connection in self.connections:
            if station1 == connection.station1 and \
               station2 == connection.station2:
                return connection
        return None

    def add_to_network(self, station1, station2):
        """Create a network of neighbouring stations"""
        if station1 in self.network.keys():
            stations = self.network[station1]
            if station2 not in stations:
                stations.append(station2)
        else:
            self.network[station1] = [station2]

    def get_directions(self, name1, name2):
        """Returns a route between two stations.  Assumes a connected, closed 
        chain of stations.  If both stations exists, then a route exists.
        If route is empty, then one or both stations do not exist."""

        # Do we have the stations in our list?
        if not self.has_station(name1) or not self.has_station(name2):
            return []

        # Stations exist, lets create the station objects and some empty
        # lists and a dictionary.  The Data structures (list and dictionary)
        # will be used to add information about if we can reach the staiton
        # as we build the route.  The list 'previous' is used to allow
        # us to reconstruct the connections.
        station1 = Station(name1)
        station2 = Station(name2)
        route = []
        reachable = []
        previous = {}

        # Check for the simple case where only need to travel one station.
        # If the destination is one station away then we are done. Append 
        # the connection to the route and stop executing instructions
        # that is, retrun from the method.  Otherwise, keep track of
        # what is reachable, and the previous station.
        neighbours = self.network[station1]
        for station in neighbours:
            if station == station2: 
                route.append(self.get_connection(station1, station2))
                return route
            else:
                reachable.append(station)
                previous[station] = station1

        # If we reach this part of the code then we are more than one 
        # station away.  We know this because in the true-branch of the 
        # if-statement above, there is a 'return' statement.  It the return
        # statement executes, we never get to this part of the code.
        # So if the computer tries to run/excute the next line of 
        # code we din execute the return statement, so we must be more than
        # one station away.
        #
        # We need to keep a list of stations to check from our current location
        next_stations = []
        next_stations.extend(neighbours)
        current_station = station1

        found_it = False  # allow early exit once route found

        # Try every station in our list (database)
        for i in range(1, len(self.stations)):
            if found_it:
                break
            tmp_next_stations = []
            for station in next_stations:
                if found_it:
                    break
                reachable.append(station)
                current_station = station
                current_neighbours = self.network[current_station]
                for neighbour in current_neighbours:
                    if found_it:
                        break
                    if neighbour == station2:
                        reachable.append(neighbour)
                        previous[neighbour] = current_station
                        found_it = True
                    elif neighbour not in reachable:
                        reachable.append(neighbour)
                        tmp_next_stations.append(neighbour)
                        previous[neighbour] = current_station
            next_stations = tmp_next_stations

        # path has been found.  Want the route to be a list of connections.
        # Initialise variable for loops
        keep_looping = True
        key_station = station2
        station = None

        # Loop over path, adding connections
        while keep_looping:
            station = previous[key_station]
            route.insert(0, self.get_connection(station, key_station))
            if station1 == station:
                keep_looping = False
            key_station = station

        return route

In [ ]:
def print_directions(route):
    """Create a text description of the subway route"""
    connection = route[0]
    station1 = connection.station1
    station2 = connection.station2
    current_line = connection.line
    previous_line = current_line

    print(f'Start out at {station1.name}.')
    print(f'Get on the {current_line} heading towards {station2.name}')

    for conneciton in route[1:]:
        current_line = connection.line
        station1 = connection.station1
        station2 = connection.station2
        if current_line == previous_line:
            print(f'Continue past {station1.name}...')
        else:
            print(f'When you get to {station1.name}, get off the {previous_line}.')
            print(f'Switch over to the {current_line}, heading towards {station2.name}.')
            previous_line = current_line

    print(f'Get off at {station2.name} and enjoy yourself!')

In [ ]:
loader = SubwayLoader()
loader.load_from_file("objectville_subway.txt")
subway = loader.subway
print_directions(subway.get_directions("AJAX Rapids", "JavaRanch"))

If you have completed building the classes then the above cell will run an initial test to get directions.  If working then you should get the following output.

    Start out at Ajax Rapids.
    Get on the Booch Line heading towards UML Walk
    When you get to UML Walk, get off the Booch Line.
    Switch over to the Wirfs-Brock Line, heading towards The Tikibean Lounge.
    Continue past The Tikibean Lounge...
    Continue past Head First Lounge...
    Continue past Objectville Diner...
    When you get to Servlet Springs, get off the Wirfs-Brock Line.
    Switch over to the Jacobson Line, heading towards Mighty Gumball, Inc..
    Continue past Mighty Gumball, Inc....
    Get off at JavaRanch and enjoy yourself!